<a href="https://colab.research.google.com/github/Centrattic/SEESZooniverse/blob/main/data_processing/parsing_Zooniverse_output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re
import numpy as np
from tqdm import tqdm
import pandas as pd

## load in csv


In [ ]:
data_df = pd.read_csv("/content/drive/MyDrive/Data_Files/zooni_output.csv")

In [ ]:
test = data_df['annotations'][0]

In [ ]:
len(data_df['annotations'])

6058

## Extracted Land, Sky, Water Presence

In [ ]:
def find_features(col):
  water_arr = [-1] * len(col)
  sky_arr = [-1] * len(col)
  land_arr = [-1] * len(col)

  for i in range(len(col)): # 0, 1, 2, 3, ... len-1
    if '"Water"' in col[i]:
      water_arr[i] = 1
    else:
      water_arr[i] = 0
    if '"Sky"' in col[i]:
      sky_arr[i] = 1
    else:
      sky_arr[i] = 0
    if '"Land"' in col[i]:
      land_arr[i] = 1
    else:
      land_arr[i] = 0

  return water_arr, sky_arr, land_arr

In [ ]:
water_col, sky_col, land_col = find_features(data_df['annotations'])

In [ ]:
data_df['water'] = water_col
data_df['sky'] = sky_col
data_df['land'] = land_col

## Extract Image Name

In [ ]:
def find_name(col):
  name_arr = [''] * len(col)

  for i in range(len(col)): # 0, 1, 2, 3, ... len-1
    img_name = re.findall(r'[0-9]+', col[i])[-1] + ".jpg"
    name_arr[i] = img_name

  return name_arr

In [ ]:
name_col = find_name(data_df['subject_data'])

In [ ]:
data_df['img_name'] = name_col

## Deleting Duplicates

In [ ]:
len(np.unique(data_df['img_name']))

5896

In [ ]:
np.random.randint

1

In [ ]:
def deleting_duplicates(df):
  unique_names = np.unique(df['img_name'])
  for i in range(len(unique_names)):
    # subset with same image name
    img_subset = df[df['img_name'] == unique_names[i]]
    if len(img_subset) > 1:
      choice = np.random.randint(0,len(img_subset))
      # drop row of choice from subset
      indxs = list(img_subset.index)
      del indxs[choice]
      # indxs now contains all indxs we want to remove
      df.drop(indxs, inplace=True)

  return df

In [ ]:
dropped_df = deleting_duplicates(data_df)

In [ ]:
dropped_df.reset_index(drop=True, inplace=True)

In [ ]:
len(np.unique(dropped_df['img_name'])) # we succeeded!

5896

In [ ]:
data_df[data_df['user_name'] == 'roayba.adhi']

In [ ]:
dropped_df.to_csv("drive/MyDrive/Data_Files/parsed_zooni_out.csv")